# Fashion-Gen Multi-Modal Classification

This notebook runs the Fashion-Gen project on Google Colab with GPU support.

## Setup Instructions
1. Upload this notebook to Google Colab
2. Run all cells in order
3. Enable GPU: Runtime → Change runtime type → GPU


## 1. Setup Environment


In [ ]:
# Install dependencies
%pip install torch torchvision numpy matplotlib Pillow scikit-learn h5py -q

# Verify GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


## 2. Clone Project from GitHub

Clone the FashionGen repository from GitHub.


In [ ]:
# Clone the repository
!git clone https://github.com/Sashahajjar/FashionGen.git

# Find the cloned project folder (handles different repo names)
import os
import sys

# The repo will be cloned to /content/FashionGen
# But let's make it flexible in case the folder name is different
possible_paths = ['/content/FashionGen', '/content/fashiongen-project']
project_path = None

for path in possible_paths:
    if os.path.exists(path):
        project_path = path
        break

# If not found, search for it
if project_path is None:
    for item in os.listdir('/content'):
        full_path = f'/content/{item}'
        if os.path.isdir(full_path) and ('fashion' in item.lower() or 'FashionGen' in item):
            project_path = full_path
            break

if project_path:
    os.chdir(project_path)
    sys.path.insert(0, project_path)
    print(f"✓ Project found at: {project_path}")
    print(f"✓ Current directory: {os.getcwd()}")
    print(f"✓ Added to Python path")
else:
    print("✗ Project folder not found. Please check the clone was successful.")


In [ ]:
# Verify project structure
import os
print("Project structure:")
for root, dirs, files in os.walk('.'):
    level = root.replace('.', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")


## 3. Download Fashion-Gen Dataset from Kaggle

The Fashion-Gen dataset is available on Kaggle: https://www.kaggle.com/datasets/bothin/fashiongen-validation/data

**Steps to download:**
1. Go to Kaggle.com and sign in
2. Go to Account → API → Create New API Token
3. Upload the `kaggle.json` file in the cell below
4. The dataset will be automatically downloaded

**Note:** If you skip this step, the project will use mock data for testing.


In [ ]:
# Install Kaggle API
%pip install kaggle -q

# ============================================================
# ⚠️  YOU MUST ADD YOUR OWN CREDENTIALS HERE! ⚠️
# ============================================================
# 
# Get your credentials from Kaggle:
# 1. Go to https://www.kaggle.com → Settings → API
# 2. Click "Create New API Token" (downloads kaggle.json)
# 3. Open kaggle.json and copy your username and key
# 4. Paste them below:
#
# ============================================================

# 👇 REPLACE THESE WITH YOUR ACTUAL CREDENTIALS 👇
KAGGLE_USERNAME = ""  # ⬅️ PASTE YOUR KAGGLE USERNAME HERE
KAGGLE_KEY = ""       # ⬅️ PASTE YOUR API TOKEN HERE (starts with KGAT_)

# ============================================================

import json
import os
from google.colab import files

print("=" * 60)
print("DIRECT DOWNLOAD FROM KAGGLE")
print("=" * 60)

# Check if credentials were pasted
if KAGGLE_USERNAME and KAGGLE_KEY:
    # Use pasted credentials
    kaggle_config = {
        "username": KAGGLE_USERNAME.strip(),
        "key": KAGGLE_KEY.strip()
    }
    print("✓ Using pasted credentials")
else:
    # Try uploading kaggle.json file instead
    print("\n📁 No credentials pasted. Upload kaggle.json file instead:")
    print("   (Or go back and paste your username and key above)")
    uploaded = files.upload()
    
    kaggle_config = None
    if uploaded:
        for filename in uploaded.keys():
            if 'kaggle' in filename.lower() and filename.endswith('.json'):
                with open(filename, 'r') as f:
                    kaggle_config = json.load(f)
                os.remove(filename)  # Remove for security
                print(f"✓ Loaded credentials from {filename}")
                break

# Set up Kaggle API if credentials provided
if kaggle_config:
    # Create .kaggle directory
    os.makedirs('/root/.kaggle', exist_ok=True)
    
    # Write kaggle.json
    with open('/root/.kaggle/kaggle.json', 'w') as f:
        json.dump(kaggle_config, f)
    
    # Set proper permissions
    os.chmod('/root/.kaggle/kaggle.json', 0o600)
    
    print("\n" + "=" * 60)
    print("✓ Kaggle API credentials configured!")
    print("\n📥 Downloading Fashion-Gen dataset from Kaggle...")
    print("   This may take a few minutes...")
    print("=" * 60)
    
    # Download dataset with better error handling
    print("\n📥 Starting download...")
    import subprocess
    result = subprocess.run(
        ['kaggle', 'datasets', 'download', '-d', 'bothin/fashiongen-validation', '-p', 'data/'],
        capture_output=True,
        text=True
    )
    
    # Print output
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print("Errors/Warnings:", result.stderr)
    
    # Check if download was successful
    import zipfile
    zip_files = [f for f in os.listdir('data/') if f.endswith('.zip')]
    
    if zip_files:
        print(f"\n✅ Download successful! Found {len(zip_files)} zip file(s)")
        for zip_file in zip_files:
            zip_path = f'data/{zip_file}'
            size_mb = os.path.getsize(zip_path) / (1024 * 1024)
            print(f"  - {zip_file} ({size_mb:.1f} MB)")
            
            print(f"\n📦 Extracting {zip_file}...")
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall('data/')
                print(f"✅ Extracted {zip_file} successfully!")
                os.remove(zip_path)
                print(f"🗑️ Removed zip file")
            except Exception as e:
                print(f"❌ Error extracting {zip_file}: {e}")
        
        # Verify .h5 files after extraction
        h5_files = [f for f in os.listdir('data/') if f.endswith('.h5')]
        if h5_files:
            print(f"\n✅ Found {len(h5_files)} .h5 file(s) after extraction:")
            for h5_file in h5_files:
                size_mb = os.path.getsize(f'data/{h5_file}') / (1024 * 1024)
                print(f"  - {h5_file} ({size_mb:.1f} MB)")
        else:
            print("\n⚠️ No .h5 files found after extraction. Checking zip contents...")
            # List contents of zip to see what's inside
            for zip_file in zip_files:
                zip_path = f'data/{zip_file}'
                try:
                    with zipfile.ZipFile(zip_path, 'r') as z:
                        print(f"\nContents of {zip_file}:")
                        for name in z.namelist()[:10]:  # Show first 10 files
                            print(f"  - {name}")
                        if len(z.namelist()) > 10:
                            print(f"  ... and {len(z.namelist()) - 10} more files")
                except:
                    pass
    else:
        print("\n❌ Download failed - no zip files found in data/")
        print(f"Return code: {result.returncode}")
        if result.returncode != 0:
            print("\n💡 Possible issues:")
            print("  1. Dataset name might be incorrect")
            print("  2. You might need to accept dataset terms on Kaggle website")
            print("  3. Check if the dataset is public and accessible")
            print("\nTry:")
            print("  - Visit: https://www.kaggle.com/datasets/bothin/fashiongen-validation/data")
            print("  - Click 'Download' button and accept terms")
            print("  - Then use Method B to upload the file manually")
    
    print("\n" + "=" * 60)
    if zip_files or h5_files:
        print("✅ Dataset download process completed!")
    else:
        print("⚠️ Dataset download did not complete successfully")
    print("=" * 60)
else:
    print("\n⚠ No credentials provided. Will use mock data for training.")
    print("\nTo download the real dataset:")
    print("  1. Get your API token from Kaggle Settings → API")
    print("  2. Paste your username and key in the variables above")
    print("  3. Run this cell again")


### Method B: Upload HDF5 File (If you already have it)

**Only use this if you already downloaded the dataset:**
- Upload the `.h5` file(s) directly below
- Or upload a zip file and it will be extracted automatically


In [ ]:
# Manual upload of HDF5 files (No API token needed!)
from google.colab import files
import os
import zipfile

print("=" * 60)
print("UPLOAD FASHION-GEN HDF5 FILES")
print("=" * 60)
print("\n📁 Click 'Choose Files' below to upload your .h5 file(s)")
print("   (Downloaded from Kaggle dataset page)")
print("\n" + "-" * 60)

# Create data directory
os.makedirs('data', exist_ok=True)

# Upload files
uploaded = files.upload()

print("\n" + "=" * 60)

if uploaded:
    # Move uploaded files to data directory
    for filename in uploaded.keys():
        # Check if it's a zip file
        if filename.endswith('.zip'):
            print(f"📦 Extracting {filename}...")
            zip_path = f'/content/{filename}'
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('data/')
            os.remove(zip_path)
            print(f"✓ Extracted {filename}")
        # Check if it's an HDF5 file
        elif filename.endswith('.h5'):
            import shutil
            shutil.move(filename, f'data/{filename}')
            print(f"✓ Moved {filename} to data/")
        else:
            # Move other files to data directory
            import shutil
            shutil.move(filename, f'data/{filename}')
            print(f"✓ Moved {filename} to data/")
    
    print("\n✓ Files uploaded successfully!")
else:
    print("⚠ No files uploaded. Will use mock data for training.")

print("=" * 60)


In [ ]:
# Verify dataset files and search for .h5 files
import os
import glob

print("=" * 60)
print("SEARCHING FOR DATASET FILES")
print("=" * 60)

# Search in multiple locations
search_paths = [
    'data/',
    '/content/data/',
    '/content/',
    '.',
    'FashionGen/data/',
    '/content/FashionGen/data/'
]

h5_files = []
zip_files = []

print("\n🔍 Searching for .h5 files...")
for path in search_paths:
    if os.path.exists(path):
        # Search for .h5 files
        for h5_file in glob.glob(os.path.join(path, '**/*.h5'), recursive=True):
            if h5_file not in h5_files:
                h5_files.append(h5_file)
        
        # Search for .zip files (might not be extracted)
        for zip_file in glob.glob(os.path.join(path, '**/*.zip'), recursive=True):
            if 'fashiongen' in zip_file.lower() and zip_file not in zip_files:
                zip_files.append(zip_file)

# Check current directory structure
print("\n📁 Current directory:", os.getcwd())
print("\n📂 Contents of data/ folder:")
if os.path.exists('data/'):
    for item in os.listdir('data/'):
        item_path = os.path.join('data', item)
        if os.path.isdir(item_path):
            print(f"  📁 {item}/")
        else:
            size_mb = os.path.getsize(item_path) / (1024 * 1024)
            print(f"  📄 {item} ({size_mb:.2f} MB)")

# Report findings
print("\n" + "=" * 60)
if h5_files:
    print(f"✅ Found {len(h5_files)} HDF5 file(s):")
    for h5_file in h5_files:
        size_mb = os.path.getsize(h5_file) / (1024 * 1024)
        print(f"  - {h5_file} ({size_mb:.1f} MB)")
    print("\n✅ Real dataset is ready for training!")
elif zip_files:
    print(f"⚠️ Found {len(zip_files)} zip file(s) that need extraction:")
    for zip_file in zip_files:
        size_mb = os.path.getsize(zip_file) / (1024 * 1024)
        print(f"  - {zip_file} ({size_mb:.1f} MB)")
    print("\n💡 The dataset might be in a zip file. Let's extract it...")
    import zipfile
    for zip_file in zip_files:
        print(f"\n📦 Extracting {zip_file}...")
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                zip_ref.extractall('data/')
            print(f"✅ Extracted {zip_file}")
            # Remove zip after extraction
            os.remove(zip_file)
        except Exception as e:
            print(f"❌ Error extracting {zip_file}: {e}")
    
    # Search again after extraction
    print("\n🔍 Searching again after extraction...")
    h5_files = []
    for h5_file in glob.glob('data/**/*.h5', recursive=True):
        h5_files.append(h5_file)
    
    if h5_files:
        print(f"✅ Now found {len(h5_files)} HDF5 file(s)!")
        for h5_file in h5_files:
            size_mb = os.path.getsize(h5_file) / (1024 * 1024)
            print(f"  - {h5_file} ({size_mb:.1f} MB)")
else:
    print("❌ No .h5 files or dataset zip files found!")
    print("\n💡 The Kaggle download might have failed or files are in a different location.")
    print("\nTry this:")
    print("  1. Check if the download completed successfully")
    print("  2. Re-run the Kaggle download cell")
    print("  3. Or manually upload the .h5 file using Method B")

# Create data directories if they don't exist
os.makedirs('data/images', exist_ok=True)
os.makedirs('data/captions', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

print("=" * 60)


### Alternative: Download Files One at a Time (More Reliable)

Download validation file first (smaller, faster), then training file separately.
This is more reliable for large files.


In [ ]:
# Download validation file FIRST (1.8 GB - smaller, more reliable)
import os
import subprocess
import zipfile
import time

print("=" * 60)
print("STEP 1: DOWNLOAD VALIDATION FILE (1.8 GB)")
print("=" * 60)
print("\n📥 Downloading: fashiongen_256_256_validation.h5")
print("   Size: 1.8 GB")
print("   Estimated time: 5-10 minutes")
print("=" * 60)

os.chdir('/content/FashionGen')
os.makedirs('data', exist_ok=True)

start_time = time.time()

# Download validation file
result = subprocess.run(
    ['kaggle', 'datasets', 'download', 'bothin/fashiongen-validation', 
     '-f', 'fashiongen_256_256_validation.h5', '-p', 'data/'],
    capture_output=True,
    text=True,
    timeout=900  # 15 minutes timeout
)

elapsed = time.time() - start_time
print(f"\n⏱️  Download took: {int(elapsed // 60)}m {int(elapsed % 60)}s")
print(f"Return code: {result.returncode}")

if result.stdout:
    print("\nOutput:", result.stdout[:300])
if result.stderr and result.returncode != 0:
    print("\nErrors:", result.stderr[:300])

# Check and extract
if os.path.exists('data/'):
    files = os.listdir('data/')
    zip_files = [f for f in files if f.endswith('.zip') and 'validation' in f.lower()]
    h5_files = [f for f in files if f.endswith('.h5') and 'validation' in f.lower()]
    
    if zip_files:
        print(f"\n✅ Found zip: {zip_files[0]}")
        print("   Extracting...")
        try:
            with zipfile.ZipFile(f'data/{zip_files[0]}', 'r') as z:
                z.extractall('data/')
            os.remove(f'data/{zip_files[0]}')
            print("   ✅ Extracted!")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    # Check final result
    files = os.listdir('data/')
    h5_files = [f for f in files if f.endswith('.h5') and 'validation' in f.lower()]
    
    if h5_files:
        size_mb = os.path.getsize(f'data/{h5_files[0]}') / (1024 * 1024)
        print(f"\n✅ SUCCESS! Validation file ready: {h5_files[0]} ({size_mb:.1f} MB)")
        print("\n💡 Now run the next cell to download the training file")
    else:
        print("\n⚠️ Validation file not found. Check errors above.")

print("=" * 60)


In [ ]:
# Download training file SECOND (14.4 GB - large, may take 30+ minutes)
import os
import subprocess
import zipfile
import time

print("=" * 60)
print("STEP 2: DOWNLOAD TRAINING FILE (14.4 GB)")
print("=" * 60)
print("\n⚠️  WARNING: This is a VERY LARGE file!")
print("   - Size: 14.4 GB")
print("   - Estimated time: 30-60 minutes")
print("   - Keep Colab open during download")
print("   - May timeout in free tier")
print("\n💡 If this fails, consider using manual download from Kaggle website")
print("=" * 60)

os.chdir('/content/FashionGen')
os.makedirs('data', exist_ok=True)

start_time = time.time()

# Download training file with longer timeout
print("\n📥 Starting download...")
print("   This will take a while - please be patient!")

result = subprocess.run(
    ['kaggle', 'datasets', 'download', 'bothin/fashiongen-validation', 
     '-f', 'fashiongen_256_256_train.h5', '-p', 'data/'],
    capture_output=True,
    text=True,
    timeout=7200  # 2 hour timeout for very large files
)

elapsed = time.time() - start_time
elapsed_min = int(elapsed // 60)
elapsed_sec = int(elapsed % 60)

print(f"\n⏱️  Download took: {elapsed_min}m {elapsed_sec}s")
print(f"Return code: {result.returncode}")

if result.returncode == -9:
    print("\n❌ Process was killed (return code -9)")
    print("   This usually means:")
    print("   - Download timed out")
    print("   - Out of memory")
    print("   - Process killed by system")
    print("\n💡 Solutions:")
    print("   1. Try manual download from Kaggle website")
    print("   2. Use Colab Pro for longer sessions")
    print("   3. Download in smaller chunks (if available)")

if result.stdout:
    print("\nOutput:", result.stdout[:500])
if result.stderr and result.returncode != 0:
    print("\nErrors:", result.stderr[:500])

# Check and extract
if os.path.exists('data/'):
    files = os.listdir('data/')
    zip_files = [f for f in files if f.endswith('.zip') and 'train' in f.lower()]
    h5_files = [f for f in files if f.endswith('.h5') and 'train' in f.lower()]
    
    if zip_files:
        zip_path = f'data/{zip_files[0]}'
        zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"\n✅ Found zip: {zip_files[0]} ({zip_size_mb:.1f} MB)")
        print("   Extracting (this may take several minutes)...")
        try:
            extract_start = time.time()
            with zipfile.ZipFile(zip_path, 'r') as z:
                z.extractall('data/')
            extract_time = time.time() - extract_start
            os.remove(zip_path)
            print(f"   ✅ Extracted in {int(extract_time // 60)}m {int(extract_time % 60)}s!")
        except Exception as e:
            print(f"   ❌ Extraction error: {e}")
    
    # Check final result
    files = os.listdir('data/')
    h5_files = [f for f in files if f.endswith('.h5')]
    
    if h5_files:
        print(f"\n✅ SUCCESS! Found {len(h5_files)} .h5 file(s):")
        total_size = 0
        for hf in h5_files:
            size_mb = os.path.getsize(f'data/{hf}') / (1024 * 1024)
            total_size += size_mb
            print(f"  - {hf} ({size_mb:.1f} MB)")
        print(f"\n📊 Total dataset size: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
        print("\n🎉 Complete dataset ready for training!")
    else:
        print("\n⚠️ Training file not found. Download may have failed.")
        print("   Check errors above or try manual download.")

print("=" * 60)


## 4. Training

Train the model with your data (or mock data for testing)


In [ ]:
# Import training script
from training.train import train

# Run training
# This will use mock data if no real data is available
train()


## 5. Evaluation

Evaluate the trained model on test data


In [ ]:
# Import evaluation script
from training.evaluate import main as evaluate

# Run evaluation
evaluate()


## 6. Inference Demo

Run inference on sample images


In [ ]:
# Import inference script
from inference.demo import demo

# Run inference demo
demo()
